In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Part 1: Import the Housing data and do feature transformations

In [ ]:
df= pd.read_csv('house_price_full.csv')
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [ ]:
X = df.copy()
# Remove target
Y = X.pop('price')

# perform a scaler transform of the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# perform log transformation of target variable (For Sandeep: Is this needed?)
Y = np.log(Y)

In [ ]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [ ]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

## Part 2: Create Model Using `keras`

![](multiple_neurons.png)

In [ ]:
from tensorflow import keras

In [ ]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            2, activation="sigmoid", input_shape=(X.shape[-1],)
        ),
        keras.layers.Dense(1, activation="linear")
    ]
)
model.summary()

```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [ ]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1) 
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [ ]:
model.fit(X,Y.values,epochs=10,batch_size=32)

2021-12-28 16:39:52.206212: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-28 16:39:52.302809: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
16/16 [==============================] - 1s 6ms/step - loss: 130.8849
Epoch 2/10
16/16 [==============================] - 0s 3ms/step - loss: 48.3066
Epoch 3/10
16/16 [==============================] - 0s 3ms/step - loss: 15.0036
Epoch 4/10
16/16 [==============================] - 0s 3ms/step - loss: 4.2597
Epoch 5/10
16/16 [==============================] - 0s 3ms/step - loss: 1.6313
Epoch 6/10
16/16 [==============================] - 0s 3ms/step - loss: 0.9751
Epoch 7/10
16/16 [==============================] - 0s 3ms/step - loss: 0.7346
Epoch 8/10
16/16 [==============================] - 0s 3ms/step - loss: 0.5919
Epoch 9/10
16/16 [==============================] - 0s 3ms/step - loss: 0.4938
Epoch 10/10
16/16 [==============================] - 0s 3ms/step - loss: 0.4205


In [ ]:
model.predict(X)[:,0]

array([13.128477 , 12.628965 , 13.178449 , 13.180457 , 13.151085 ,
       12.900153 , 13.024193 , 13.047293 , 13.174511 , 13.168509 ,
       13.166689 , 13.005024 , 13.178679 , 13.0899315, 12.836474 ,
       13.108126 , 13.154926 , 12.962559 , 13.132423 , 13.10496  ,
       13.180997 , 13.168523 , 13.169441 , 13.043495 , 13.167471 ,
       13.18181  , 13.177605 , 13.125883 , 12.860254 , 13.163628 ,
       13.168126 , 13.017628 , 13.159059 , 13.025801 , 12.830246 ,
       13.08225  , 12.876249 , 13.175127 , 13.179413 , 13.176378 ,
       13.155876 , 13.168619 , 12.891278 , 13.170714 , 12.990326 ,
       13.101732 , 13.179699 , 13.178449 , 13.118698 , 12.91412  ,
       13.047293 , 13.131868 , 13.154086 , 13.172497 , 13.004393 ,
       13.152577 , 13.127048 , 12.626102 , 13.125765 , 13.08344  ,
       12.91412  , 13.156879 , 13.043495 , 13.168009 , 13.005024 ,
       12.90601  , 13.162054 , 13.075506 , 13.194309 , 13.181587 ,
       13.079155 , 13.13918  , 13.11602  , 13.177332 , 13.1271